# Introduction

This project will focus on exploring the capabilities of Bayesian optimization, specifically employing BayBE, in the discovery of novel corrosion inhibitors for materials design. Initially, we will work with a randomly chosen subset from a comprehensive database of electrochemical responses of small organic molecules. Our goal is to assess how Bayesian optimization can speed up the screening process across the design space to identify promising compounds. We will compare different strategies for incorporating alloy information, while optimizing the experimental parameters with respect to the inhibitive performance of the screened compounds.

# Initizalization

Loading libraries and data files:

In [1]:
import pandas as pd
import numpy as np

df_AA2024 = pd.read_excel('data/filtered_AA2024.xlsx')
# df_AA1000 = pd.read_excel('data/filtered_AA1000.xlsx')
# df_Al = pd.read_excel('data/filtered_Al.xlsx')

In [2]:
print(df_AA2024.describe())

           Time_h          pH  Inhib_Concentrat_M  Salt_Concentrat_M  \
count  611.000000  611.000000          611.000000         611.000000   
mean   135.801964    6.342062            0.006808           0.145450   
std    201.683867    2.529080            0.014059           0.200575   
min      0.500000    0.000000            0.000010           0.000000   
25%     24.000000    4.000000            0.000500           0.010000   
50%     24.000000    7.000000            0.001000           0.100000   
75%    144.000000    7.000000            0.003000           0.100000   
max    672.000000   10.000000            0.100000           0.600000   

        Efficiency  
count   611.000000  
mean     26.736841  
std     288.788317  
min   -4834.000000  
25%      30.000000  
50%      58.000000  
75%      87.950000  
max     100.000000  


In [3]:
print(df_AA2024.head())

                                    SMILES  Time_h    pH  Inhib_Concentrat_M  \
0             COCCOC(=O)OCSc1nc2c(s1)cccc2    24.0   4.0               0.001   
1             COCCOC(=O)OCSc1nc2c(s1)cccc2    24.0  10.0               0.001   
2            Cc1ccc(c(c1)n1nc2c(n1)cccc2)O    24.0   4.0               0.001   
3            Cc1ccc(c(c1)n1nc2c(n1)cccc2)O    24.0  10.0               0.001   
4  Clc1ccc(cc1)CC[C@](C(C)(C)C)(Cn1cncn1)O    24.0   4.0               0.001   

   Salt_Concentrat_M  Efficiency  
0                0.1         0.0  
1                0.1         0.0  
2                0.1        30.0  
3                0.1        30.0  
4                0.1        30.0  


# Data Processing

### Extract all unique SMILES values into dictionary

In [4]:
unique_SMILES = df_AA2024.SMILES.unique()

def list_to_dict(input_list):
    return {item: item for item in input_list}

smiles_dict =list_to_dict(unique_SMILES)

# Data Anaylsis

# Bayesian Optimization

## Search Space

In [5]:
from baybe.parameters import NumericalDiscreteParameter, NumericalContinuousParameter
from baybe.searchspace import SearchSpace

parameters = [
NumericalDiscreteParameter(
    name="Time (h)",
    values=np.arange(1, 25, 1)
    # tolerance = 0.004, assume certain experimental noise for each parameter measurement?
),
NumericalDiscreteParameter(
    name="pH",
    values=np.arange(-1, 15.1, 0.1)
    # tolerance = 0.004
    ),  
NumericalDiscreteParameter( # Set this as continuous, the values seem quite small?
    name="Inhibitor Concentration (M)",
    values=np.arange(0, 0.1, 0.01), # Remove data outliers like 0.1?
    # tolerance = 0.004
    ),
NumericalDiscreteParameter(
    name="Salt Concentration (M)",
    values=np.arange(0, 2.01, 0.01),
    # tolerance = 0.004
    )
]

/home/vscode/.local/lib/python3.10/site-packages/baybe/telemetry.py:222: UserWarning: WARNING: BayBE Telemetry endpoint https://public.telemetry.baybe.p.uptimize.merckgroup.com:4317 cannot be reached. Disabling telemetry. The exception encountered was: ConnectionError, HTTPConnectionPool(host='verkehrsnachrichten.merck.de', port=80): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7f8bf60907f0>: Failed to resolve 'verkehrsnachrichten.merck.de' ([Errno -2] Name or service not known)"))
  warnings.warn(


In [6]:
from baybe.parameters import SubstanceParameter

encoding_choice = ["MORDRED", "RDKIT", "MORGAN_FP"]

SubstanceParameter(
    name="Inhibitor",
    data=smiles_dict,
    encoding=encoding_choice[2], # Which is better?
    decorrelate=0.7,  # Change threshold to avoid overfitting?
)

SubstanceParameter(name='Inhibitor', data={'COCCOC(=O)OCSc1nc2c(s1)cccc2': 'COCCOC(=O)OCSc1nc2c(s1)cccc2', 'Cc1ccc(c(c1)n1nc2c(n1)cccc2)O': 'Cc1ccc(c(c1)n1nc2c(n1)cccc2)O', 'Clc1ccc(cc1)CC[C@](C(C)(C)C)(Cn1cncn1)O': 'Clc1ccc(cc1)CC[C@](C(C)(C)C)(Cn1cncn1)O', 'On1nnc2c1cccc2': 'On1nnc2c1cccc2', 'c1ncn[nH]1': 'c1ncn[nH]1', 'Sc1n[nH]cn1': 'Sc1n[nH]cn1', 'S[C]1NC2=C[CH]C=NC2=N1': 'S[C]1NC2=C[CH]C=NC2=N1', 'S=c1[nH]c2c([nH]1)nccn2': 'S=c1[nH]c2c([nH]1)nccn2', 'Sc1ncc[nH]1': 'Sc1ncc[nH]1', 'C=CC(=O)OCCOC(=O)OCCSc1ncccn1': 'C=CC(=O)OCCOC(=O)OCCSc1ncccn1', 'CCSc1nnc(s1)N': 'CCSc1nnc(s1)N', 'CSc1nnc(s1)N': 'CSc1nnc(s1)N', 'Cc1ccc2c(c1)nc([nH]2)S': 'Cc1ccc2c(c1)nc([nH]2)S', 'OC(=O)CS': 'OC(=O)CS', 'Sc1nc2c([nH]1)cccc2': 'Sc1nc2c([nH]1)cccc2', 'OC(=O)c1ccccc1S': 'OC(=O)c1ccccc1S', 'S=c1sc2c([nH]1)cccc2': 'S=c1sc2c([nH]1)cccc2', 'OC(=O)c1cccnc1S': 'OC(=O)c1cccnc1S', 'Sc1ncccn1': 'Sc1ncccn1', 'c1ccc(nc1)c1ccccn1': 'c1ccc(nc1)c1ccccn1', 'Sc1nnc(s1)S': 'Sc1nnc(s1)S', 'Nc1cc(S)nc(n1)N': 'Nc1cc(S)nc(n1

These calculations will typically result in 500 to 1500 numbers per molecule. **To avoid detrimental effects on the surrogate model fit, we reduce the number of descriptors via decorrelation before using them.** For instance, the decorrelate option in the example above specifies that only descriptors with a correlation lower than 0.7 to any other descriptor will be kept. This usually reduces the number of descriptors to 10-50, depending on the specific items in data.

### Custom descriptors

In [ ]:
"""
The encoding concept introduced above is generalized by the CustomParameter. Here, the user is expected to provide their own descriptors for the encoding.

Take, for instance, a parameter that corresponds to the choice of a polymer. Polymers are not well represented by the small molecule descriptors utilized in the SubstanceParameter. 
Still, one could provide experimental measurements or common metrics used to classify polymers:
from baybe.parameters import CustomDiscreteParameter

# Create or import new dataframe containing custom descriptors

descriptors = pd.DataFrame(
    {
        "Glass_Transition_TempC": [20, -71, -39],
        "Weight_kDalton": [120, 32, 241],
    },
    index=["Polymer A", "Polymer B", "Polymer C"],  # put labels in the index
)

CustomDiscreteParameter(
    name="Polymer",
    data=descriptors,
    decorrelate=True,  # optional, uses default correlation threshold = 0.7?
)
""" 

In [7]:
searchspace = SearchSpace.from_product(parameters)

In [12]:
print(searchspace)

Search Space
         
 Search Space Type: DISCRETE
 
 Discrete Search Space
              
  Discrete Parameters
                            Name                        Type  Num_Values Encoding
  0                     Time (h)  NumericalDiscreteParameter          24     None
  1                           pH  NumericalDiscreteParameter         161     None
  2  Inhibitor Concentration (M)  NumericalDiscreteParameter          10     None
  3       Salt Concentration (M)  NumericalDiscreteParameter         201     None
              
  Experimental Representation
           Time (h)    pH  Inhibitor Concentration (M)  Salt Concentration (M)
  0             1.0  -1.0                         0.00                    0.00
  1             1.0  -1.0                         0.00                    0.01
  2             1.0  -1.0                         0.00                    0.02
  ...           ...   ...                          ...                     ...
  7766637      24.0  15.0           

## Objective

In [8]:
from baybe.targets import NumericalTarget
from baybe.objective import Objective

target = NumericalTarget(
    name="Efficiency",
    mode="MAX",
)
objective = Objective(mode="SINGLE", targets=[target])

## Recommender

In [9]:
from baybe.recommenders import RandomRecommender, SequentialGreedyRecommender
from baybe.surrogates import GaussianProcessSurrogate

available_surr_models = [
    "GaussianProcessSurrogate", 
    "BayesianLinearSurrogate",
    "MeanPredictionSurrogate",
    "NGBoostSurrogate",
    "RandomForestSurrogate"
]

available_acq_functions = [
    "qPI",  # q-Probability Of Improvement
    "qEI",  # q-Expected Improvement
    "qUCB", # q-upper confidence bound with beta of 1.0
]

# Defaults anyway
SURROGATE_MODEL = GaussianProcessSurrogate()
ACQ_FUNCTION = "qEI" # q-Expected Improvement, only q-fuctions are available for batch_size > 1

seq_greedy_recommender = SequentialGreedyRecommender(
        surrogate_model=SURROGATE_MODEL,
        acquisition_function_cls=ACQ_FUNCTION,
        hybrid_sampler="Farthest", # find more details in the documentation
        sampling_percentage=0.3, # should be relatively low
        allow_repeated_recommendations=False,
        allow_recommending_already_measured=False,
    )

# Campaign

In [10]:
from baybe.strategies import TwoPhaseStrategy
from baybe import Campaign

strategy = TwoPhaseStrategy(
    initial_recommender = RandomRecommender(),  # Initial recommender
    # Doesn't matter since I already have training data, BUT CAN BE USED FOR BENCHMARKING
    recommender = seq_greedy_recommender,  # Bayesian model-based optimization
    switch_after=1  # Switch to the model-based recommender after 1 batches = immediately
)

campaign = Campaign(searchspace, objective, strategy)

/home/vscode/.local/lib/python3.10/site-packages/baybe/strategies/deprecation.py:26: DeprecationWarning: 'TwoPhaseStrategy' is deprecated and will be removed in a future version. Please use 'recommenders.TwoPhaseMetaRecommender' class instead.
  warnings.warn(


In [11]:
print(campaign)

Campaign
         
 Meta Data
 Batches Done: 0
 Fits Done: 0
 
 Search Space
          
  Search Space Type: DISCRETE
  
  Discrete Search Space
               
   Discrete Parameters
                             Name                        Type  Num_Values Encoding
   0                     Time (h)  NumericalDiscreteParameter          24     None
   1                           pH  NumericalDiscreteParameter         161     None
   2  Inhibitor Concentration (M)  NumericalDiscreteParameter          10     None
   3       Salt Concentration (M)  NumericalDiscreteParameter         201     None
               
   Experimental Representation
            Time (h)    pH  Inhibitor Concentration (M)  Salt Concentration (M)
   0             1.0  -1.0                         0.00                    0.00
   1             1.0  -1.0                         0.00                    0.01
   2             1.0  -1.0                         0.00                    0.02
   ...           ...   ...        

### Get recommendations

In [ ]:
new_rec = campaign.recommend(batch_size=1) # TEST with different batch sizes for optimal performance
print("\n\nRecommended experiments: ")
print(new_rec.to_markdown())

In [ ]:
# Get and input efficiency value from Excel table, for specific SMILES component first, 
# then for the closest values of the rest of the parameters

new_rec["Efficiency"] = [0.1]
campaign.add_measurements(new_rec)

print("\n\nRecommended experiments with measured values: ")
print(new_rec.to_markdown())

In [ ]:
new_new_rec = campaign.recommend(batch_size=1) # TEST with different batch sizes for optimal performance
print("\n\nRecommended experiments: ")
print(new_new_rec.to_markdown())

In [ ]:
print(campaign)

### Merge all results into a dataframe

In [ ]:
results = pd.concat([new_rec, new_new_rec]) # etc.
print("\n\nAll experiments with measured values: ")
print(results.to_markdown())

# Benchmarking

# Transfer Learning

https://emdgroup.github.io/baybe/examples/Transfer_Learning/basic_transfer_learning.html

https://emdgroup.github.io/baybe/userguide/transfer_learning.html

https://emdgroup.github.io/baybe/userguide/simulation.html

https://emdgroup.github.io/baybe/examples/Backtesting/impute_mode.html